# 40. 係り受け解析結果の読み込み（形態素）

# 41. 係り受け解析結果の読み込み（文節・係り受け）

# 42. 係り元と係り先の文節の表示

# 43. 名詞を含む文節が動詞を含む文節に係るものを抽出

# 44. 係り受け木の可視化

# 45. 動詞の格パターンの抽出

# 46. 動詞の格フレーム情報の抽出

# 47. 機能動詞構文のマイニング

# 48. 名詞から根へのパスの抽出

# 49. 名詞間の係り受けパスの抽出